# Trabalho II - PLN

Diego Araujo Giovanini        11201811750  
Karolayne de Queiroz Campos   11201810126  

A ideia do segundo trabalho de PLN é utilizar 3 técnicas aprendidas durante o curso em apenas um notebook.  
O dataset escolhido foi um dataset do kaggle de super heróis, com mais de 81 variaveis que dão características aos super heróis até mesmo a história deles.

Através desse dataset vamos realizar a sumarização das histórias de cada super herói.  
Após a sumarização utilizaremos nayvi bayes e Regressão Logística para classificação de cada super herói tentando predizer qual o possível criador dele.  
E por fim será realizada uma similaridade de textos entre histórias de super heróis para tentar encontrar aqueles que tem histórias compartilhadas ou são do mesmo universo.

## Imports

Importando tudo que será utilizado ao longo do projeto

In [308]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import re
import nltk
import string
import numpy as np
import heapq

from nltk.tokenize import wordpunct_tokenize
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score

import spacy

nltk.download('punkt')
nltk.download('stopwords')

pd.set_option('display.max_rows', 999)

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\diego\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\diego\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [375]:
df = pd.read_csv('superheroes_nlp_dataset.csv')

## Funções Genéricas

In [376]:
def tabela_resumo(df):
    print(f"Formato dataset: {df.shape}")
    summario = pd.DataFrame(df.dtypes, columns=['dtypes'])
    summario = summario.reset_index()
    summario['Nome'] = summario['index']
    summario = summario[['Nome', 'dtypes']]
    summario['Missing'] = df.isnull().sum().values
    summario['Unicos'] = df.nunique().values
    summario['Primeiro Valor'] = df.loc[0].values
    summario['Segundo Valor'] = df.loc[1].values
    return summario

## Análise Descritiva dos dados

Existem muitas colunas nesse dataset, como vamos utilizar apenas para NLP, vamos deixar apenas as colunas que importam

In [378]:
df.shape

(1450, 81)

In [379]:
df.head(2)

,name,real_name,full_name,overall_score,history_text,powers_text,intelligence_score,strength_score,speed_score,durability_score,...,has_flight,has_accelerated_healing,has_weapons_master,has_intelligence,has_reflexes,has_super_speed,has_durability,has_stamina,has_agility,has_super_strength
0,3-D Man,"Delroy Garrett, Jr.","Delroy Garrett, Jr.",6,"Delroy Garrett, Jr. grew up to become a track ...",NaN,85,30,60,60,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
1,514A (Gotham),Bruce Wayne,NaN,10,He was one of the many prisoners of Indian Hil...,NaN,100,20,30,50,...,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0


In [380]:
df = df[['name', 'real_name', 'history_text', 'creator', 'teams', 'powers_text']]

In [381]:
tabela_resumo(df)

Formato dataset: (1450, 6)


,Nome,dtypes,Missing,Unicos,Primeiro Valor,Segundo Valor
0,name,object,2,1448,3-D Man,514A (Gotham)
1,real_name,object,149,1068,"Delroy Garrett, Jr.",Bruce Wayne
2,history_text,object,90,1349,"Delroy Garrett, Jr. grew up to become a track ...",He was one of the many prisoners of Indian Hil...
3,creator,object,139,41,Marvel Comics,DC Comics
4,teams,object,0,419,"['Annihilators', 'Asgardians', 'Avengers', 'Ne...",[]
5,powers_text,object,364,1061,NaN,NaN


Como existem muitos valores missing, a forma de tratamento vai ser desconsiderar esses casos do estudo.  

In [382]:
df = df.dropna().reset_index().drop(columns='index')

In [383]:
tabela_resumo(df)

Formato dataset: (830, 6)


,Nome,dtypes,Missing,Unicos,Primeiro Valor,Segundo Valor
0,name,object,0,830,A-Bomb,Abe Sapien
1,real_name,object,0,698,Richard Milhouse Jones,Abraham Sapien
2,history_text,object,0,821,"Richard ""Rick"" Jones was orphaned at a young ...","Sapien began life as Langdon Everett Caul, a ..."
3,creator,object,0,26,Marvel Comics,Dark Horse Comics
4,teams,object,0,321,"['Teen Brigade', 'Ultimate Fantastic Four', 'U...",['Bureau for Paranormal Research and Defense']
5,powers_text,object,0,812,"On rare occasions, and through unusual circu...",Abe is a humanoid amphibious creature. He has...


## Criando Resumo

In [384]:
df['resume'] = df['history_text'] + ' Powers: ' + df['powers_text']

## Sumarização Automática de Textos

Vamos utilizar o método no qual frases com maior frequencia tendem a representar o texto, assim vamos realizar o calculo da frequencia das palavras e após isso vamos pegas as sentenças com maior frequencia das palavras pré calculadas, assim vamos conseguir medir a frequencia das sentenças e escolher as N mais relevantes.

### Tratando o Resumo

In [385]:
#Removendo quebras de linha
df['resume_treated'] = df['resume'].apply(lambda x: re.sub(r'\n', '', x))

In [386]:
#Removendo stopwords e pontuacoes
stopwords = nltk.corpus.stopwords.words('english')
pontuacoes = string.punctuation
print(f"As stopwords são: {stopwords}\n")
print(f"As pontuações são: {pontuacoes}")

def pre_processamento(texto):
    texto_formatado = texto.lower()
    tokens = []
    texto_limpo = []
    for token in nltk.word_tokenize(texto_formatado):
        tokens.append(token)
    
    for token in tokens:
        if token not in stopwords and token not in pontuacoes:
            texto_limpo.append(token)
    
    texto_limpo_unificado = ' '.join([str(elemento) for elemento in texto_limpo])
    
    return texto_limpo_unificado

As stopwords são: ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same

In [387]:
df['resume_treated'] = df['resume_treated'].apply(lambda x : pre_processamento(x))

In [388]:
#Exemplo de texto não limpo
df['resume'][0]

' Richard "Rick" Jones was orphaned at a young age, and after being expelled from several orphanages for disciplinary reasons he was placed into a state institution called Tempest Town. A troubled and rebellious youth, Jones soon came to the attention of the institution\'s chief administrator who smashed his guitar, a gift from his late father, and then had him severely thrashed. Soon afterward, Jones ran away from the institution. He spent the first half of his teens drifting from town to town throughout the Southwest, trying to avoid the juvenile authorities, and doing menial work when he could get it.  At age 16, he got his driver\'s license and managed to save enough money to buy a used car. Overhearing a teenager dare a friend to ride out with him on the desert where it was rumored an atomic bomb was going to be tested, Jones offered to take him upon the challenge. He drove his car out to the test site to discover his challenger was too timid to show up. Dr. Robert Bruce Banner, d

In [389]:
#Exemplo de texto limpo
df['resume_treated'][0]

"richard `` rick '' jones orphaned young age expelled several orphanages disciplinary reasons placed state institution called tempest town troubled rebellious youth jones soon came attention institution 's chief administrator smashed guitar gift late father severely thrashed soon afterward jones ran away institution spent first half teens drifting town town throughout southwest trying avoid juvenile authorities menial work could get age 16 got driver 's license managed save enough money buy used car overhearing teenager dare friend ride desert rumored atomic bomb going tested jones offered take upon challenge drove car test site discover challenger timid show dr. robert bruce banner designer gamma bomb tested learned someone ventured onto test site believing countdown delayed ran desert warn back banner managed throw jones protective trench bomb detonated bombarded gamma radiation radiation triggered mutagenic change banner causing turn raging superhuman hulk feeling responsible banner

### Cálculo de Frequencia

In [390]:
#Cálculando a frequencia das palavras

def calculo_frequencia(df):
    freq_palavras = nltk.FreqDist(nltk.word_tokenize(df.resume_treated))
    freq_max = max(freq_palavras.values())
    
    for palavra in freq_palavras.keys():
        freq_palavras[palavra] = (freq_palavras[palavra]/freq_max)
    
    sent_tkn = nltk.sent_tokenize(df.resume)
    pesos_sentencas = {}
    for sentenca in sent_tkn:
        for palavra in sentenca.lower().split():
            if palavra in freq_palavras.keys():
                if sentenca not in pesos_sentencas.keys():
                    pesos_sentencas[sentenca]= freq_palavras[palavra]
                else:
                    pesos_sentencas[sentenca] += freq_palavras[palavra]
    
    #Escolhemos as 10 sentenças com maior peso                
    return ' '.join(heapq.nlargest(10, pesos_sentencas, key=pesos_sentencas.get))

In [391]:
df['resume_final'] = df.apply(calculo_frequencia, axis=1)

In [129]:
#Exemplo de Sumarização x Texto real
print(df.resume[0])
print('\n')
print(df.resume_final[0])

 Richard "Rick" Jones was orphaned at a young age, and after being expelled from several orphanages for disciplinary reasons he was placed into a state institution called Tempest Town. A troubled and rebellious youth, Jones soon came to the attention of the institution's chief administrator who smashed his guitar, a gift from his late father, and then had him severely thrashed. Soon afterward, Jones ran away from the institution. He spent the first half of his teens drifting from town to town throughout the Southwest, trying to avoid the juvenile authorities, and doing menial work when he could get it.  At age 16, he got his driver's license and managed to save enough money to buy a used car. Overhearing a teenager dare a friend to ride out with him on the desert where it was rumored an atomic bomb was going to be tested, Jones offered to take him upon the challenge. He drove his car out to the test site to discover his challenger was too timid to show up. Dr. Robert Bruce Banner, desi

## Classificação

Para a classificação será utilizado o texto resumido, que é resultado da sumarização anterior.  
Porém ainda será realizada as tecnicas de stemização, remoção de stopwords, pontuações e transformação para minusculo antes da classificação, para melhorar o desempenho do algoritmo, e para que os algoritmos consigam entender vamos vetorizar os documentos utilizando TF-IDF. 

A ideia é conseguir classificar os super heróis entre os diferentes criadores.
Para isso vamos comparar 2 modelos diferentes, Regressão Logistica e Naive Bays através da precision, recall.

### Avaliando balanceamento das classes

In [249]:
df_clf = df[['resume_final', 'creator']]

In [300]:
#Temos um problema desbalanceado onde a maioria dos heróis estão localizados na Marvel e na DC, com isso
#vamos avaliar precision e Recall, visto que acuracia não é indicada para problemas desbalanceados

#Outro ponto é que estamos trabalhando com um problema multiclasse, então na hora do calculo da precision e recall
#vamos fazer um calculo balanceado por classe.

df_clf.groupby('creator').count().sort_values(by='resume_final', ascending= False)

,resume_final,stems
creator,,
Marvel Comics,446,446
DC Comics,270,270
Lego,22,22
Shueisha,19,19
Dark Horse Comics,18,18
NBC - Heroes,13,13
George Lucas,7,7
Ubisoft,6,6
Team Epic TV,4,4


### Tratando o Texto

In [250]:
#Removendo pontuações e stopwords, transformando em minusculo, stemizando e convertendo em tokens
stm = PorterStemmer()

def preproc_tokenizer(df):
    
    # quebra o documento em  tokens
    words = wordpunct_tokenize(df.resume_final) 
    
    # converte palavras para minusculo
    words = [word.lower() for word in words]
    
    # remove as stopwords
    words = [word for word in words if word not in stopwords]
    
    # remove pontuacao
    words = [word for word in words if word not in pontuacoes]
    
    words = [stm.stem(word) for word in words]

    return(words)

In [251]:
df_clf['stems'] = df.apply(preproc_tokenizer, axis=1)

<ipython-input-251-863e81f82823>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clf['stems'] = df.apply(preproc_tokenizer, axis=1)


In [252]:
df_clf.head()

,resume_final,creator,stems
0,"When the Hulk returned to Earth, Jones attempt...",Marvel Comics,"[hulk, return, earth, jone, attempt, calm, tel..."
1,"As a result, Abe learns of Edith Howard, Langd...",Dark Horse Comics,"[result, abe, learn, edith, howard, langdon, c..."
2,While she was recovering from radiation sickne...,Marvel Comics,"[recov, radiat, sick, caus, exposur, gamma, ra..."
3,Abra Kadabra was able to spontaneously cause w...,DC Comics,"[abra, kadabra, abl, spontan, caus, water, cre..."
4,"While Abraxas drew nearer, the walls of dimens...",Marvel Comics,"[abraxa, drew, nearer, wall, dimension, space,..."


In [253]:
#Quebrando em treino e teste
train_corpus, test_corpus = train_test_split(df_clf, test_size=0.3, random_state=777)

In [254]:
#Vetorizando com TF-IDF
vectorizer = TfidfVectorizer()

train_stems = vectorizer.fit_transform([" ".join(doc) for doc in train_corpus['stems']])
test_stems = vectorizer.transform([" ".join(doc) for doc in test_corpus['stems']])

In [302]:
#Testando os 2 modelos
#É possível notar que o naive bayes foi melhor que a regressão logistica, tanto na precision quanto no recall, sendo então
#a melhor escolha para essa classificação

algoritms = {'Regressão Logística' : LogisticRegression(), 'Naive Bayes': GaussianNB()}

for name, alg in algoritms.items():
    alg.fit(train_stems.toarray(),train_corpus['creator'])
    predictions = alg.predict(test_stems.toarray())
    print("Recall do: {0} {1:.2f}%".format(name, recall_score(test_corpus['creator'],predictions, average='weighted')*100))
    print("Precision do: {0} {1:.2f}%".format(name, precision_score(test_corpus['creator'],predictions, average='weighted')*100))

Recall do: Regressão Logística 80.32%
Precision do: Regressão Logística 72.84%
Recall do: Naive Bayes 83.13%
Precision do: Naive Bayes 81.60%


## Similaridade de Texto

Por fim vamos realizar uma similaridade de texto para tentar identificar a similaridade de uma história com as demais e ver se é possível predizer se os heróis são do mesmo criador.  
A ideia é achar um threshold no qual seja possível medir o recall e a precision.

In [362]:
#Então vamos tentar fazer isso para apenas um herói e tentar achar a similaridade da história dele com outros herois.
#o herói escolhido foi o Superman (2006) que é da DC Comics
df[df.name=='Superman (2006)']

,name,real_name,history_text,creator,teams,powers_text,resume,resume_treated,resume_final
722,Superman (2006),Kal-El,"As far as is known, the early history of Kal-E...",DC Comics,[],"Like all Kryptonians, Superman develops superh...","As far as is known, the early history of Kal-E...",far known early history kal-el superman return...,"As far as is known, the early history of Kal-E..."


In [311]:
#Carregando o Englis model
nlp = spacy.load("en_core_web_sm")

In [354]:
#Cálculando similaridade do Superman (2006) contra todos os outros heróis
superman_resume = nlp(df[df.name=='Superman (2006)']['resume_treated'].reset_index()['resume_treated'][0])

In [373]:
lista_similaridade = []
for res in df[df.name!='Superman (2006)']['resume_treated'].to_list():
    lista_similaridade.append(round(superman_resume.similarity(nlp(res)),2))

In [392]:
df = df[df.name!='Superman (2006)'].join(pd.DataFrame({'Similaridade':lista_similaridade}))

In [393]:
#Para facilitar os calculos de precisão e recall vamos fazer um agrupamento one to others, agrupando DC Comics x Others
df['DC_COMICS'] = df.creator.apply(lambda x: 1 if x=='DC Comics' else 0)

In [452]:
df.head(2)

,name,real_name,history_text,creator,teams,powers_text,resume,resume_treated,resume_final,Similaridade,DC_COMICS
0,A-Bomb,Richard Milhouse Jones,"Richard ""Rick"" Jones was orphaned at a young ...",Marvel Comics,"['Teen Brigade', 'Ultimate Fantastic Four', 'U...","On rare occasions, and through unusual circu...","Richard ""Rick"" Jones was orphaned at a young ...",richard `` rick '' jones orphaned young age ex...,"When the Hulk returned to Earth, Jones attempt...",0.958,0
1,Abe Sapien,Abraham Sapien,"Sapien began life as Langdon Everett Caul, a ...",Dark Horse Comics,['Bureau for Paranormal Research and Defense'],Abe is a humanoid amphibious creature. He has...,"Sapien began life as Langdon Everett Caul, a ...",sapien began life langdon everett caul victori...,"As a result, Abe learns of Edith Howard, Langd...",0.961,0


In [442]:
df_aux = pd.DataFrame(columns=['Corte', 'Acum. Total', 'Acum. VP', 'Precision', 'Recall'])
tot_1 = df[(df.DC_COMICS == 1) & (df.Similaridade.isna() == False)].DC_COMICS.count()

for corte in df[df.Similaridade.isna() == False]['Similaridade'].unique():
    acum_tot = df[df.Similaridade >= corte].DC_COMICS.count()
    acum_vp = df[df.Similaridade >= corte].DC_COMICS.sum()
    prec = acum_vp/acum_tot
    rec = acum_vp/tot_1
    df_aux = df_aux.append(pd.DataFrame({'Corte':corte, 'Acum. Total':acum_tot, 'Acum. VP':acum_vp, 'Precision':prec, 'Recall':rec}, index=[1]))

In [444]:
#Com a tabela abaixo podemos ter uma noção de threshold para definir um ponto de corte para o % de similaridade
#de forma a tentar utilizar a similaridade de texto como um classificador, porém
#vemos que ele nem se compara a um método de classificação como Regressão Logistica ou 
#Naive Bayes, sua precision é muito mais baixa, quando igualado o mesmo recall dos modelos de classificação
#temos uma precisão em torno de 32%.

df_aux.sort_values(by='Corte', ascending =False).reset_index().drop(columns='index')

,Corte,Acum. Total,Acum. VP,Precision,Recall
0,0.983,1,0,0.000000,0.000000
1,0.978,2,0,0.000000,0.000000
2,0.977,4,0,0.000000,0.000000
3,0.975,7,1,0.142857,0.003731
4,0.974,8,1,0.125000,0.003731
5,0.973,9,1,0.111111,0.003731
6,0.972,10,1,0.100000,0.003731
7,0.971,13,3,0.230769,0.011194
8,0.970,17,3,0.176471,0.011194
9,0.969,23,4,0.173913,0.014925
